In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
# Set the input directory
flower_dataset_directory = "./data/Flowers/"
flower_category_foldername = os.listdir(flower_dataset_directory)

# Set the output directory
output_dir = './data/downscaled_images/'

# Iterate over the images
for folder in flower_category_foldername:
    image_filenames = os.listdir(flower_dataset_directory+folder)
    for filename in image_filenames:
        # Load the image
        image = cv2.imread(flower_dataset_directory + folder + "/" + filename)

        # Downscale the image using linear interpolation
        downscaled_image = cv2.resize(
            image, (224, 224), interpolation=cv2.INTER_LINEAR)

        # Save the downscaled image to a file
        folder_name = folder.replace(" ", "_")  # remove spaces from folder name
        output_filename = os.path.join(output_dir, folder_name, filename)
        os.makedirs(os.path.dirname(output_filename), exist_ok=True)  # create the subdirectories
        cv2.imwrite(output_filename, downscaled_image)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# index_group_name values
train_data_groups = ["Babi", "Calimerio", "Chrysanthemum", "Hydrangeas", "Lisianthus", "Pingpong", "Rosy", "Tana"]

epochs = 50
batch_size = 1


def extract_features(group, model, model_name):
    # empty array to store flower_id
    flower_ids = []
    datagen = ImageDataGenerator(rescale=1. / 255)
    
    # image subfolers path
    dir_path = "./data/downscaled_images/"+group+'/'
    # image data augmentation
    generator = datagen.flow_from_directory(
        dir_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    # add files names as flower_id array
    for i in generator.filenames:
        flower_ids.append(i[(i.find("/")+1):i.find(".")])
    extracted_features = model.predict(
        generator, len(generator.filenames) // batch_size)
    
    extracted_features = extracted_features.reshape(
        (len(generator.filenames), -1))
    
    
    # save extracted features
    np.save(open('./ResNet_{group}_{model_name}_features.npy'.format(
        group=group, model_name=model_name), 'wb'), extracted_features)
    np.save(open('./ResNet_{group}_{model_name}_feature_flower_ids.npy'.format(
        group=group, model_name=model_name), 'wb'),
            np.array(flower_ids))

In [ ]:
from keras import applications

resnet_model = applications.ResNet50(include_top=False)
  
for image_group in train_data_groups:
    extract_features(image_group,resnet_model, "ResNet50" )

In [ ]:
vgg16_model = applications.VGG16(include_top=False)

for image_group in train_data_groups:
    extract_features(image_group,vgg16_model, "VGG16" )

In [ ]:
# ResNet50 generated features
babi_extracted_features = np.load('./ResNet_Babi_ResNet50_features.npy')
babi_Flowerids = np.load('./ResNet_Babi_ResNet50_feature_flower_ids.npy')
calimerio_extracted_features = np.load(
    './ResNet_Calimerio_ResNet50_features.npy')
calimerio_Flowerids = np.load('./ResNet_Calimerio_ResNet50_feature_flower_ids.npy')
chrysanthemum_extracted_features = np.load(
    './ResNet_Chrysanthemum_ResNet50_features.npy')
chrysanthemum_Flowerids = np.load(
    './ResNet_Chrysanthemum_ResNet50_feature_flower_ids.npy')
hydrangeas_extracted_features = np.load(
    './ResNet_Hydrangeas_ResNet50_features.npy')
hydrangeas_Flowerids = np.load(
    './ResNet_Hydrangeas_ResNet50_feature_flower_ids.npy')
lisianthus_extracted_features = np.load('./ResNet_Lisianthus_ResNet50_features.npy')
lisianthus_Flowerids = np.load(
    './ResNet_Lisianthus_ResNet50_feature_flower_ids.npy')
pingpong_extracted_features = np.load('./ResNet_Pingpong_ResNet50_features.npy')
pingpong_Flowerids = np.load(
    './ResNet_Pingpong_ResNet50_feature_flower_ids.npy')
rosy_extracted_features = np.load('./ResNet_Rosy_ResNet50_features.npy')
rosy_Flowerids = np.load(
    './ResNet_Rosy_ResNet50_feature_flower_ids.npy')
tana_extracted_features = np.load('./ResNet_Tana_ResNet50_features.npy')
tana_Flowerids = np.load(
    './ResNet_Tana_ResNet50_feature_flower_ids.npy')


VGG16_babi_extracted_features = np.load('./ResNet_Babi_VGG16_features.npy')
VGG16_babi_Flowerids = np.load('./ResNet_Babi_VGG16_feature_flower_ids.npy')
VGG16_calimerio_extracted_features = np.load(
    './ResNet_Calimerio_VGG16_features.npy')
VGG16_calimerio_Flowerids = np.load('./ResNet_Calimerio_VGG16_feature_flower_ids.npy')
VGG16_chrysanthemum_extracted_features = np.load(
    './ResNet_Chrysanthemum_VGG16_features.npy')
VGG16_chrysanthemum_Flowerids = np.load(
    './ResNet_Chrysanthemum_VGG16_feature_flower_ids.npy')
VGG16_hydrangeas_extracted_features = np.load(
    './ResNet_Hydrangeas_VGG16_features.npy')
VGG16_hydrangeas_Flowerids = np.load(
    './ResNet_Hydrangeas_VGG16_feature_flower_ids.npy')
VGG16_lisianthus_extracted_features = np.load('./ResNet_Lisianthus_VGG16_features.npy')
VGG16_lisianthus_Flowerids = np.load(
    './ResNet_Lisianthus_VGG16_feature_flower_ids.npy')
VGG16_pingpong_extracted_features = np.load('./ResNet_Pingpong_VGG16_features.npy')
VGG16_pingpong_Flowerids = np.load(
    './ResNet_Pingpong_VGG16_feature_flower_ids.npy')
VGG16_rosy_extracted_features = np.load('./ResNet_Rosy_VGG16_features.npy')
VGG16_rosy_Flowerids = np.load(
    './ResNet_Rosy_VGG16_feature_flower_ids.npy')
VGG16_tana_extracted_features = np.load('./ResNet_Tana_VGG16_features.npy')
VGG16_tana_Flowerids = np.load(
    './ResNet_Tana_VGG16_feature_flower_ids.npy')

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(babi_extracted_features[0].reshape(224, 448),  vmax=.3,  cmap="YlGnBu")

plt.show()

In [ ]:
# display a single image
def display_image(path):
    img = Image.open(path)
    display(img)

In [ ]:
import os
import csv

input_dir = "data/Flowers"
output_file = "metadata.csv"

subfolders = os.listdir(input_dir)

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(['flower_id', 'flower_type', 'flower_filename'])

    id_counter = 1
    for i, subfolder in enumerate(subfolders):
        image_filenames = os.listdir(os.path.join(input_dir, subfolder))

        for filename in image_filenames:
            writer.writerow([id_counter, subfolder, filename])
            id_counter += 1

In [ ]:
flowers = pd.read_csv("metadata.csv")

def get_extracted_features_resnet(flower_id):
    if (flowers[flowers['flower_id'] == flower_id].empty):
        print(
            flowers[flowers['flower_id'] == flower_id])
        return "No item matched input Id", []
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Babi"):
        extracted_features = babi_extracted_features
        flower_ids = babi_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Calimerio"):
        extracted_features = calimerio_extracted_features
        flower_ids = calimerio_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Chrysanthemum"):
        extracted_features = chrysanthemum_Flowerids
        flower_ids = chrysanthemum_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Hydrangeas"):
        extracted_features = hydrangeas_extracted_features
        flower_ids = hydrangeas_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Lisianthus"):
        extracted_features = lisianthus_extracted_features
        flower_ids = lisianthus_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Pingpong"):
        extracted_features = pingpong_extracted_features
        flower_ids = pingpong_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Rosy"):
        extracted_features = rosy_extracted_features
        flower_ids = rosy_Flowerids
    else:
        extracted_features = tana_extracted_features
        flower_ids = tana_Flowerids
    return extracted_features, flower_ids

def get_extracted_features_VGG16(flower_id):
    if (flowers[flowers['flower_id'] == flower_id].empty):
        print(
            flowers[flowers['flower_id'] == flower_id])
        return "No item matched input Id"
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Babi"):
        extracted_features = VGG16_babi_extracted_features
        flower_ids = VGG16_babi_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Calimerio"):
        extracted_features = VGG16_calimerio_extracted_features
        flower_ids = VGG16_calimerio_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Chrysanthemum"):
        extracted_features = VGG16_chrysanthemum_Flowerids
        flower_ids = VGG16_chrysanthemum_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Hydrangeas"):
        extracted_features = VGG16_hydrangeas_extracted_features
        flower_ids = VGG16_hydrangeas_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Lisianthus"):
        extracted_features = VGG16_lisianthus_extracted_features
        flower_ids = VGG16_lisianthus_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Pingpong"):
        extracted_features = VGG16_pingpong_extracted_features
        flower_ids = VGG16_pingpong_Flowerids
    elif (flowers[flowers['flower_id'] == flower_id]['flower_type'].values[0] == "Rosy"):
        extracted_features = VGG16_rosy_extracted_features
        flower_ids = VGG16_rosy_Flowerids
    else:
        extracted_features = tana_extracted_features
        flower_ids = VGG16_tana_Flowerids
    return extracted_features, flower_ids

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def get_similar_products_cnn(flower_id, num_results, model):

    extracted_features, flower_ids = get_extracted_features_VGG16(
        flower_id) if model == "VGG16" else get_extracted_features_resnet(flower_id)


    # list of all flowers
    FlowerIds = list(flower_ids)

    print(FlowerIds, "Adssad")

    # index of the input flower_id in the flower list
    input_flower_id_idx = FlowerIds.index(flower_id)

    ''' 
    calculate cosine sim score 
    from the feature of the 
    input image with others features  
    '''
    cosine_sim = cosine_similarity(
        extracted_features, extracted_features[input_flower_id_idx].reshape(1, -1))
    
    # sort indices and sim score to get the most similar ones
    indices = np.argsort(cosine_sim.flatten())[::-1][0:num_results]
    similarities = np.sort(cosine_sim.flatten())[::-1][0:num_results]

    print("="*20, "input flower details", "="*20)
    
    # path of input image
    input_row = flowers[flowers['flower_id'] == int(
        FlowerIds[indices[0]])]
    
    # path of input image
    input_img_path = "downscaled_images/" + \
        input_row['flower_type'].values[0]+"/" + \
        "0"+str(input_row["flower_id"].values[0])+".jpg"
    display_image(input_img_path)
    print('Flower File name: ', input_row['flower_file_name'])
    print('Flower Type: ', input_row['flower_type'])
    print("\n", "="*20, "Recommended flowers", "="*20)
    
    # display top similar images with highest cosine similarity score
    for i in range(1, len(indices)):
        # get metatdata
        rows = flowers[flowers['flower_id'] == int(
            FlowerIds[indices[i]])]
        # display images
        for indx, row in rows.iterrows():
            rec_img_path = "downscaled_images/" + \
                row['flower_type'] + \
                str(row["flower_id"])+".jpg"
            display_image(rec_img_path)
            print('Flower Title: ', row['flower_file_name'])
            print('Flower Type: ', row['flower_type'])
            
            print('Cosine similarity score:', similarities[i])

In [ ]:
get_similar_products_cnn("babi_1", 10, "ResNet50")